# LangChain

The sourse idea is here https://youtu.be/aywZrzNaKjs

## Установка библиотек. Сервисные функции

In [ ]:
!pip -q install langchain==0.0.137
!pip -q install openai

In [ ]:
!pip -q install --upgrade tiktoken

In [ ]:
# !pip -q install openai chromadb

In [ ]:
import os
import ipywidgets as widgets

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class MyTools():
  def __init__(self):
    pass

  @classmethod
  def set_key(cls):
      password_input = widgets.Password(
          description='Введите OpenAi API key:',
          layout=widgets.Layout(width='500px'),
          style={'description_width': 'initial', 'white-space': 'pre-wrap', 'overflow': 'auto'})
      login_button = widgets.Button(description='Авторизация')
      output = widgets.Output()

      def on_button_clicked(_):
          with output:
              api_key = password_input.value
              os.environ["OPENAI_API_KEY"] = api_key
              print(f'{bcolors.OKGREEN}{bcolors.BOLD}Ключ сохранен!{bcolors.ENDC}')
              password_input.layout.display = 'none'
              login_button.layout.display = 'none'

      login_button.on_click(on_button_clicked)
      display(widgets.VBox([password_input, login_button, output]))



## Установка OpenAI API key

In [ ]:
tools = MyTools()
MyTools.set_key()

## 1. Базовый запрос к OpenAI

In [ ]:
# Выполнить базовый запрос
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
llm("Объясни large language models (LLM) одним предложением")

In [ ]:
# импортировать схему для сообщений чата и ChatGPT чтобы запрашивать модели chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
messages = [
    SystemMessage(content="Ты специалист по Artificial Intelligence (AI)"),
    HumanMessage(content="Напишите код Python, который обучает нейронную сеть на смоделированных данных.")
]
response=chat(messages)

print(response.content,end='\n')

## 2. Prompt шаблон

In [ ]:
# Импортировать prompt и определить PromptTemplate

from langchain import PromptTemplate

template = """
Вы опытный специалист по данным с опытом построения моделей глубокого обучения.
Объясните концепцию {concept} в двух строках.
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

In [ ]:
prompt

In [ ]:
# Вызывать LLM с PromptTemplate

llm(prompt.format(concept="autoencoder"))

In [ ]:
llm(prompt.format(concept="regularization"))

## 3. Chains

In [ ]:
# Импортировать LLMChain и определить  chain с LLM и prompt как параметр.

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Выполнить chain только для указаной входной переменной.
print(chain.run("autoencoder"))

In [ ]:
# Определить второй prompt

second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Включите описание концепции {ml_concept} и объясните мне это, как будто мне 5 лет в 500 словах.",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
# Define a sequential chain using the two chains above: the second chain takes the output of the first chain as input

from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("autoencoder")
print(explanation)

## 4. Embeddings and VectorStores

In [ ]:
# Import utility for splitting up texts and split up the explanation given above into document chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 0,
)

texts = text_splitter.create_documents([explanation])

In [ ]:
texts

In [ ]:
texts[0].page_content

In [ ]:
# Import and instantiate OpenAI embeddings
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model_name="ada")

In [ ]:
# Turn the first text chunk into a vector with the embedding
import tiktoken
query_result = embeddings.embed_query(texts[0].page_content)
print(query_result)